In [1]:
import os
import shutil
import argparse
import logging
from pathlib import Path
import zipfile

def setup_logging():
    """Configure logging for the script"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

def parse_arguments():
    """Parse command line arguments"""
    parser = argparse.ArgumentParser(description='Move files from Archive directories to an archive drive')
    parser.add_argument('initial_base', help='Initial base directory to search for Archive folders')
    parser.add_argument('archive_base', help='Archive base directory where files will be moved to')
    return parser.parse_args()

def find_archive_directories(base_dir):
    """Find all directories named 'Archive' within the base directory"""
    archive_dirs = []
    for root, dirs, _ in os.walk(base_dir):
        for dir_name in dirs:
            if dir_name == 'Archive':
                archive_dirs.append(os.path.join(root, dir_name))
    return archive_dirs

def get_relative_path(full_path, base_path):
    """Get the relative path of a directory from the base directory"""
    return os.path.relpath(full_path, base_path)

def ensure_directory_exists(directory):
    """Ensure a directory exists, creating it if necessary"""
    os.makedirs(directory, exist_ok=True)

def zip_datz_files(directory):
    """Zip all .datz files in the given directory into one archive"""
    datz_files = sorted([f for f in os.listdir(directory) if f.lower().endswith('.datz')])
    if not datz_files:
        return None

    # Build zip name based on first and last datz filenames
    first_name = os.path.splitext(datz_files[0])[0]
    last_name = os.path.splitext(datz_files[-1])[0]

    # Take the last 15 characters of last_name, strip any leading underscore
    tail = last_name[-15:].lstrip('_')

    zip_name = f"{first_name}-{tail}.zip"
    zip_path = os.path.join(directory, zip_name)

    # Create zip archive
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for f in datz_files:
            full_path = os.path.join(directory, f)
            zipf.write(full_path, arcname=f)
            os.remove(full_path)
            logging.info(f"Added and removed: {full_path}")

    logging.info(f"Created zip archive: {zip_path}")
    return zip_path

def move_files(archive_dir, initial_base, archive_base):
    """Move (or zip and move) files from the source Archive directory to the target Archive directory"""
    rel_path = get_relative_path(archive_dir, initial_base)
    target_dir = os.path.join(archive_base, rel_path)
    ensure_directory_exists(target_dir)

    # Zip .datz files first
    zip_path = zip_datz_files(archive_dir)
    files_moved = 0

    # Move remaining files
    for filename in os.listdir(archive_dir):
        source_file = os.path.join(archive_dir, filename)
        target_file = os.path.join(target_dir, filename)
        if os.path.isfile(source_file):
            try:
                shutil.move(source_file, target_file)
                logging.info(f"Moved: {source_file} -> {target_file}")
                files_moved += 1
            except Exception as e:
                logging.error(f"Failed to move {source_file}: {e}")

    # Move the ZIP (if created) last
    if zip_path and os.path.exists(zip_path):
        try:
            target_zip = os.path.join(target_dir, os.path.basename(zip_path))
            shutil.move(zip_path, target_zip)
            logging.info(f"Moved zip archive: {zip_path} -> {target_zip}")
            files_moved += 1
        except Exception as e:
            logging.error(f"Failed to move zip archive {zip_path}: {e}")

    return files_moved

def main(initial_base=None, archive_base=None):
    """Main function to process the directory structure and move files"""
    setup_logging()

    if initial_base is None:
        args = parse_arguments()
        initial_base = os.path.abspath(args.initial_base)
        archive_base = os.path.abspath(args.archive_base)

    logging.info(f"Starting archive move from {initial_base} to {archive_base}")

    if not os.path.isdir(initial_base):
        logging.error(f"Initial directory {initial_base} does not exist")
        return 1

    if not os.path.isdir(archive_base):
        logging.warning(f"Archive directory {archive_base} does not exist. Creating it.")
        ensure_directory_exists(archive_base)

    archive_dirs = find_archive_directories(initial_base)
    logging.info(f"Found {len(archive_dirs)} Archive directories")

    total_files_moved = 0
    for archive_dir in archive_dirs:
        files_moved = move_files(archive_dir, initial_base, archive_base)
        total_files_moved += files_moved
        logging.info(f"Processed {archive_dir}: moved {files_moved} files")

    logging.info(f"Archive move completed. Total files moved: {total_files_moved}")
    return 0


In [ ]:
from_arch = 'C:\\Users\\rhansen\\Documents\\Python\\SPMs'
to_arch = 'G:\\Python\\SPM_Data_Archive'

main(from_arch, to_arch)

2025-11-19 13:46:45 - INFO - Starting archive move from C:\Users\rhansen\Documents\Python\SPMs to G:\Python\SPM_Data_Archive
